In [1]:
from transformers import pipeline
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch
import os

#print(torch.cuda.device_count())
#os.environ["CUDA_VISIBLE_DEVICES"] = "2,3"

#モデル読み込み
model_name = "meta-llama/Llama-2-7b-chat-hf"

model_name = "tokyotech-llm/Swallow-MS-7b-v0.1"
model_name = "cyberagent/open-calm-7b"

model_name="Local-Novel-LLM-project/Vecteus-v1"
#model_name="nvidia/Llama3-ChatQA-1.5-8B"

model_name="../model/------llm-models-hf-2ja1_part_inst_all_lr_1e-5"
model_name="../model/------llm-models-hf-2ja1_part_inst_all_lr_1e-5"
model_name="../X_merge/merged_models/0507test"
model_name="../../llm/models/hf/step41300",
model_name="../model/------llm-models-hf-step41300_inst_all-parquet_lr_1e-4"
model_name="../model/------llm-models-hf-step41300_inst_all-parquet_lr_1e-4"

tokenizer = AutoTokenizer.from_pretrained(model_name)


/home/hatakeyama/miniconda3/envs/llmeval2/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers
/home/hatakeyama/miniconda3/envs/llmeval2/lib/python3.11/site-packages/transformers/convert_slow_tokenizer.py:560: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [17]:
t_list=[
"\n\n### 応答:\n",
"\n\n### 応答:\n「",
"\n\n### 応答:\nああ",
"\n\n### 応答:\naaa",
"1\n\n### 応答:\naaa",
"あ\n\n### 応答:\naaa",
"」\n\n### 応答:\naaa",
]
t_list=[
    "",
    "a \n a",
"a\n   ",
"\n\n### 指示:\n",
"\n\n### 指示:\n「",
"\n\n### 指示:\nああ",
"\n\n### 指示:\naaa",
"1\n\n### 指示:\naaa",
"あ\n\n### 指示:\naaa",
"」\n\n### 指示:\naaa",
]
for t in t_list:
    print(tokenizer.encode(t,add_special_tokens=False),tokenizer.tokenize(t))


[] []
[355, 8, 355] ['▁a', '\n', '▁a']
[355, 8] ['▁a', '\n']
[8, 8, 5092, 272, 3994, 327, 8] ['\n', '\n', '▁###', '▁', '指示', ':', '\n']
[8, 8, 5092, 272, 3994, 327, 8, 337] ['\n', '\n', '▁###', '▁', '指示', ':', '\n', '▁「']
[8, 8, 5092, 272, 3994, 327, 8, 8722] ['\n', '\n', '▁###', '▁', '指示', ':', '\n', '▁ああ']
[8, 8, 5092, 272, 3994, 327, 8, 272, 55841] ['\n', '\n', '▁###', '▁', '指示', ':', '\n', '▁', 'aaa']
[272, 279, 8, 8, 5092, 272, 3994, 327, 8, 272, 55841] ['▁', '1', '\n', '\n', '▁###', '▁', '指示', ':', '\n', '▁', 'aaa']
[4543, 8, 8, 5092, 272, 3994, 327, 8, 272, 55841] ['▁あ', '\n', '\n', '▁###', '▁', '指示', ':', '\n', '▁', 'aaa']
[272, 285, 8, 8, 5092, 272, 3994, 327, 8, 272, 55841] ['▁', '」', '\n', '\n', '▁###', '▁', '指示', ':', '\n', '▁', 'aaa']


In [3]:
tokenizer.encode(t)

[272, 285, 8, 8, 5092, 272, 3994, 327, 8, 272, 55841, 2]

In [4]:
model = AutoModelForCausalLM.from_pretrained(model_name, 
                                            #quantization_config=bnb_config, 
                                            device_map="auto",
                                            )


Loading checkpoint shards: 100%|██████████| 4/4 [00:09<00:00,  2.47s/it]


In [5]:
#model.load_adapter(model_path)

In [6]:
#層の表示
for name, param in model.named_parameters():
    print(name)
    if "gate" in name:
        print("***",name)

model.embed_tokens.weight
model.layers.0.self_attn.q_proj.weight
model.layers.0.self_attn.k_proj.weight
model.layers.0.self_attn.v_proj.weight
model.layers.0.self_attn.o_proj.weight
model.layers.0.mlp.gate_proj.weight
*** model.layers.0.mlp.gate_proj.weight
model.layers.0.mlp.up_proj.weight
model.layers.0.mlp.down_proj.weight
model.layers.0.input_layernorm.weight
model.layers.0.post_attention_layernorm.weight
model.layers.1.self_attn.q_proj.weight
model.layers.1.self_attn.k_proj.weight
model.layers.1.self_attn.v_proj.weight
model.layers.1.self_attn.o_proj.weight
model.layers.1.mlp.gate_proj.weight
*** model.layers.1.mlp.gate_proj.weight
model.layers.1.mlp.up_proj.weight
model.layers.1.mlp.down_proj.weight
model.layers.1.input_layernorm.weight
model.layers.1.post_attention_layernorm.weight
model.layers.2.self_attn.q_proj.weight
model.layers.2.self_attn.k_proj.weight
model.layers.2.self_attn.v_proj.weight
model.layers.2.self_attn.o_proj.weight
model.layers.2.mlp.gate_proj.weight
*** mode

In [7]:
def perplexity(model, tokenizer, text) -> torch.Tensor:
    tokenized_input = tokenizer.encode(
        text, add_special_tokens=False, return_tensors="pt"
    ).to(model.device)
    with torch.inference_mode():
        output = model(tokenized_input, labels=tokenized_input)
    ppl = torch.exp(output.loss)
    return ppl.item()



In [8]:
pipe=pipeline('text-generation',model=model,tokenizer=tokenizer, 
              max_new_tokens=50, 
              repetition_penalty=1.2,
              temperature=0.6,
              #repetition_penalty=1.,
              #temperature=0.1,
              #top_p=1.0,
              #top_k=0.
              )


In [14]:
question_template="以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。\n\n### 指示:\n"
answer_template="\n\n### 応答:\n"

def gen_prompt(q):
    return f"{question_template}{q}{answer_template}"
    return question_template+q+answer_template

gen_prompt("こんにちは")

'以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。\n\n### 指示:\nこんにちは\n\n### 応答:'

In [15]:
questions=[
"こんにちは",
"元気ですか?",
"いい天気ですね",
"明日の天気は?",
"計算してください 1+1はいくつですか｡",
"フィボナッチ数列を生成するpythonのコードを作成してください",
"ドラえもんの友達はだれですか?",
"レイリー散乱とはなんですか",
"hello!",
"今の天気は晴れです｡この結果をjsonで出力してください｡ ",
"純粋理性批判はたぬきに理解できますか?",
]
questions=[
    "アスキーアートを出力して",
]

for question in questions:
    #question_template="以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。<SEP>指示<SEP>"
    #inp=f"{question_template}{question}<SEP>応答<SEP>"
    inp=gen_prompt(question)
    #inp=f"質問: {question}回答: "
    print("--------------------")
    print(question)
    res=pipe(inp)[0]["generated_text"][len(inp):]
    print(res)
    print(tokenizer.encode(res))

--------------------
アスキーアートを出力して
 1. 画像ファイル名(例えば「swf」)2. 文字列3. テキスト4. 音声5. 動画6. 音楽7. 写真8. 映像9.
[272, 279, 284, 272, 609, 667, 2808, 538, 290, 731, 703, 292, 293, 30781, 285, 289, 283, 284, 272, 1762, 1845, 299, 284, 272, 5800, 311, 284, 272, 7190, 307, 284, 272, 561, 609, 328, 284, 272, 1268, 336, 284, 272, 682, 332, 284, 272, 1243, 667, 321, 284, 2]


# ベンチマーク問題での応答評価

In [ ]:
pipe=pipeline('text-generation',model=model,tokenizer=tokenizer, 
              max_new_tokens=50, 
              repetition_penalty=1.,
              temperature=0.1,
              top_p=1.0,
              top_k=0.
              )

In [ ]:
#ベンチマーク問題への応答性の軽い評価
import pandas as pd
jaster_path="bench_data/jaster.txt"
with open(jaster_path,"r") as f:
    input_list=f.readlines()

for question in input_list:
    #question_template="以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。<SEP>指示<SEP>"
    #inp=f"{question_template}{question}<SEP>応答<SEP>"
    inp=gen_prompt(question)
    #inp=f"質問: {question}回答: "
    print("--------------------")
    print(question)
    print("出力")
    print(pipe(inp)[0]["generated_text"][len(inp):])

In [ ]:
jmt_path="bench_data/jmt.csv"
df=pd.read_csv(jmt_path)
input_list=df["input"].to_list()
for question in input_list:
    #question_template="以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。<SEP>指示<SEP>"
    #inp=f"{question_template}{question}<SEP>応答<SEP>"
    inp=gen_prompt(question)
    #inp=f"質問: {question}回答: "
    print("--------------------")
    print(question)
    print("出力")
    print(pipe(inp)[0]["generated_text"][len(inp):])